# Optimising compute with concurrent IO in OpenCL

With many iterative processes there is a need to get information **off** the device at regular intervals. Up to this point we have been transferring data off the compute device **after** kernel execution. Also, the routines to read information from device buffers have thus far been used in a blocking manner, that is the program pauses while the read occurs. Most compute devices have the ability to transfer data **while** kernels are being executed. This means IO transfers can take place during compute and may in some instances **take place entirely** during kernel execution. For the cost of additional programming complexity, significant compute savings can be obtained, as the following diagram illustrates:

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/optimising_io.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: The difference between sequential and concurrent IO.</figcaption>
</figure>

## Concurrent IO is enabled with multiple command queues

The **key to leveraging concurrent IO** is to have one command queue for the kernels and one or more command queues for moving data. Then IO operations can take place largely independently of the compute operations. OpenCL Events and the [clFinish](https://registry.khronos.org/OpenCL/sdk/3.0/docs/man/html/clFinish.html) command can help establish dependencies between command queues. Non-blocking IO calls may also help with concurrency.

## Example with the 2D wave equation

The [scalar wave equation](https://en.wikipedia.org/wiki/Wave_equation) adequately describes a number of different wave phenomena. If **U** is a 2D grid storing the amplitude of the wave at every location (the wavefield), **V** is a 2D grid storing velocity, and **t** is time, then 2D waves propagate according to the formula,

$$\frac{\partial^2 \textbf{U}}{{\partial t}^2}=\textbf{V}^2 \left (\frac{\partial^2 \textbf{U}}{{\partial x_{0}}^2}+\frac{\partial^2 \textbf{U}}{{\partial x_{1}}^2} \right)+f(t)$$

where $x_0$ and $x_1$ are spatial directions and $f(t)$ is a forcing term. If $\Delta t$ is the time step a second-order finite-difference approximation to the time derivative is given in terms of the amplitude at timesteps $\textbf{U}_{0}, \textbf{U}_{1}$ and $\textbf{U}_{2}.$ 

$$\frac{\partial^2 \textbf{U}}{{\partial t}^2} \approx \frac{1}{\Delta t^2} \left ( \textbf{U}_{0} -2 \textbf{U}_{1}+\textbf{U}_{2} \right ) $$

Replace $\frac{\partial^2 \textbf{U}}{{\partial t}^2}$ with $\frac{1}{\Delta t^2} \left( \textbf{U}_{0} -2 \textbf{U}_{1}+\textbf{U}_{2} \right )$ and solve for $\textbf{U}_{2}$.

$$\textbf{U}_{2} \approx 2 \textbf{U}_{1} - \textbf{U}_{0} + \Delta t^2\textbf{V}^2 \left (\frac{\partial^2 \textbf{U}_{1}}{{\partial x_{0}}^2}+\frac{\partial^2 \textbf{U}_{1}}{{\partial x_{1}}^2} \right)+f_{1}$$

This is an iterative formula to generate the amplitude at the next timestep $\textbf{U}_2$ if we know the present ampltiude $\textbf{U}_{1}$ and past amplitude $\textbf{U}_{0}.$ We also use finite difference approximations for the spatial derivatives, and express the spatial derivatives as a matrix multiplied by $\textbf{U}_{1}$, but this complexity is unnecessary to show here. All we need to know is that the next timestep is a function ${\textbf{F}}$ of the present and past timesteps, the velocity, and the forcing term.

$$\textbf{U}_{2}=\textbf{F}(\textbf{U}_0, \textbf{U}_1, \textbf{V}, f_{1})$$

> In geophysics we usually use a [Ricker Wavelet](https://wiki.seg.org/wiki/Dictionary:Ricker_wavelet) for the forcing term $f$, and usually inject that wavelet into one cell within the grid as time progresses.

### Kernel implementation

In [kernels.c](kernels.c) is a kernel called **wave2d_4o** that implements the function **F**. OpenCL buffers store $\textbf{U}_{0}, \textbf{U}_{1}, \textbf{U}_{2}$, and $\textbf{V}$ on the compute device.

```C++
// Kernel to solve the wave equation with fourth-order accuracy in space
__kernel void wave2d_4o (
        __global float* U0,
        __global float* U1,
        __global float* U2,
        __global float* V,
        unsigned int N0,
        unsigned int N1,
        float dt2,
        float inv_dx02,
        float inv_dx12,
        // Position, frequency, and time for the
        // wavelet injection
        unsigned int P0,
        unsigned int P1,
        float pi2fm2t2) {    

    // U2, U1, U0, V is of size (N0, N1)
    size_t i0=get_global_id(1); // Slowest dimension
    size_t i1=get_global_id(0); // Fastest dimension
    
    // Required padding and coefficients for spatial finite difference
    const int pad_l=2, pad_r=2, ncoeffs=5;
    float coeffs[ncoeffs] = {-0.083333336f, 1.3333334f, -2.5f, 1.3333334f, -0.083333336f};
    
    // Limit i0 and i1 to the region of U2 within the padding
    i0=min(i0, (size_t)(N0-1-pad_r));
    i1=min(i1, (size_t)(N1-1-pad_r));
    i0=max((size_t)pad_l, i0);
    i1=max((size_t)pad_l, i1);
    
    // Position within the grid as a 1D offset
    long offset=i0*N1+i1;
    
    // Temporary storage
    float temp0=0.0f, temp1=0.0f;
    float tempV=V[offset];
    
    // Calculate the Laplacian
    #pragma unroll
    for (long n=0; n<ncoeffs; n++) {
        // Stride in dim0 is N1        
        temp0+=coeffs[n]*U1[offset+(n*(long)N1)-(pad_l*(long)N1)];
        // Stride in dim1 is 1
        temp1+=coeffs[n]*U1[offset+n-pad_l];
    }
    
    // Calculate the wavefield U2 at the next timestep
    U2[offset]=(2.0f*U1[offset])-U0[offset]+((dt2*tempV*tempV)*(temp0*inv_dx02+temp1*inv_dx12));
    
    // Inject the forcing term at coordinates (P0, P1)
    if ((i0==P0) && (i1==P1)) {
        U2[offset]+=(1.0f-2.0f*pi2fm2t2)*exp(-pi2fm2t2);
    }
    
}
```

### Problem setup

For this problem we create the 2D grid as a square box of size $(N0,N1)=(256,256)$. The velocity is uniform at 343m/s. This is approximately the speed of sound in air. Then we use a Ricker wavelet as a forcing term to 'let off a firework' in the middle of the box and run a number of timesteps to see how a sound wave propagates in the box. 

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:80%">
    <img style="vertical-align:middle" src="../images/wave2d_problem.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Problem setup for the 2D wave equation.</figcaption>
</figure>

At each timestep the kernel **wave2d_4o** is used to update the solution to the next timestep. Old wavefields that are no longer needed are recycled for efficiency.

The code below writes the velocity array to disk, ready to be read in by the wave codes below.

In [2]:
%matplotlib widget

import os
import sys
import numpy as np
import subprocess
from ipywidgets import widgets
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

sys.path.insert(0, os.path.abspath("../include"))

import py_helper

float_type = np.float32

defines=py_helper.load_defines("mat_size.hpp")

# Velocity of the medium
vel=343.0

# Make up the velocity and first two timesteps
V=np.ones((defines["N0"], defines["N1"]), dtype=float_type)*vel

# write files to disk
V.tofile("array_V.dat")

### Sequential (synchronous) IO solution

In [wave2d_sync.cpp](wave2d_sync.cpp) we use an array of three OpenCL buffers to represent the wavefield at timesteps (0,1,2). A single command queue is used for both kernel execution and IO.

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/sequential_io.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Sequential IO solution.</figcaption>
</figure>

#### Make and run the application.

In [3]:
!make

make: Nothing to be done for 'all'.


In [4]:
subprocess.run([os.path.join(os.getcwd(),"wave2d_sync.exe"), "-gpu"])

	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
dt=0.001166, Vmax=343.000000
dt=0.00116618, fm=34.3, Vmax=343, dt2=1.35998e-06
The synchronous calculation took 84009 milliseconds.


CompletedProcess(args=['/home/toby/Pelagos/Projects/OpenCL_Course/course_material/L9_IO_Optimisation/wave2d_sync.exe', '-gpu'], returncode=0)

#### Application trace

The script **make_traces.sh** produces traces in the **tau** folder.

In [5]:
!./make_traces.sh

	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
dt=0.001166, Vmax=343.000000
dt=0.00116618, fm=34.3, Vmax=343, dt2=1.35998e-06
device id: -2042467264.
command id: 94555955408576.
vendor id: 0.
Got a bogus start! 2 .TAU application
The synchronous calculation took 100352 milliseconds.
/opt/tau/2.31.1/x86_64/bin/tau_merge -m tau.edf -e events.0.edf events.0.edf events.0.edf tautrace.0.0.0.trc tautrace.0.0.1.trc tautrace.0.0.2.trc tau.trc
tautrace.0.0.0.trc: 18909 records read.
tautrace.0.0.1.trc: 6 records read.
tautrace.0.0.2.trc: 8326 records read.
	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
dt=0.001166, Vmax=343.000000
dt=0.00116618, fm=34.3, Vmax=343, dt2=1.35998e-06
device id: -315371040.
command id: 94179739064784.
vendor id: 0.
Got a bogus start! 2 .TAU application
dev

Using google chrome type in the address [chrome://tracing](chrome://tracing) to open the tracing utility. If you load the file **tau/trace_sync.json** you should see something like this. The IO occurs after each kernel execution using the same command queue.

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/synchronous_io.png"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Sequential IO solution.</figcaption>
</figure>

### Concurrent (asynchronous) IO solution

In [wave2d_async.cpp](wave2d_async.cpp) is the solution for concurrent IO. The goal is to use **multiple command queues** so that while a  command queue is executing a kernel, the other is working on IO.

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/concurrent_io.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Concurrent IO solution.</figcaption>
</figure>

#### Concurrent access to buffers is undefined

In OpenCL it is **undefined behaviour** to read from a Buffer (using another command queue) at the same time as a kernel is using it. During construction of this module I found that reading from an OpenCL Buffer while it is being used by a kernel resulted in the instability of the solution or even a program crash. Therefore, we can safely read from Buffers when no kernel is using them. Our kernel needs access to wavefields at timesteps $\textbf{U}_{0}, \textbf{U}_{1}, \textbf{U}_{2}$, therefore they are **not** safe to copy, but wavefields at earlier timesteps e.g $\textbf{U}_{-2}, \textbf{U}_{-1}$ **are** safe to copy.

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:30%">
    <img style="vertical-align:middle" src="../images/wavefields.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Wavefields that are ok to copy.</figcaption>
</figure>

#### Using Events to aid with concurrency

The solution to enable concurrent IO is to have an array of at least four OpenCL buffers. We choose an array of **nscratch=5** buffers to allow extra time for copies to finish. Associated with the buffers is an array of five command queues for IO. Every queued OpenCL operation can depend on a list of OpenCL Events and produce one OpenCL event. Therefore we also create an array of five Events to make sure that IO waits for the kernel execution to finish. A separate compute queue is used just for the kernel. During each iteration **n** of the time loop then:

1. Use **[clFinish](https://registry.khronos.org/OpenCL/sdk/3.0/docs/man/html/clFinish.html)** waits for all activity on command_queue[(n+2)%nscratch] to complete.
1. Submit the kernel to solve for U[(n+2)%nscratch], place an event at Events[n%nscratch]
1. Use the command_queue[(n-1)%nscratch] to copy the buffer at U[(n-1)%nscratch] back to host, depend on Events[(n-1)%nscratch] to make sure the kernel has finished.

The following diagram shows how the dependencies play out with Events and command queues during a single iteration.

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:30%">
    <img style="vertical-align:middle" src="../images/wavefields_concurrent.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Compute and IO queues during an iteration.</figcaption>
</figure>

The code for the iterations is produced here.

```C++
    for (int n=0; n<NT; n++) {
        
        // Wait for the previous copy command to finish
        h_errchk(
            clFinish(command_queues[(n+2)%nscratch]),
            "Waiting for all previous things to finish"
        );
        
        // Get the wavefields
        U0 = buffers_U[n%nscratch];
        U1 = buffers_U[(n+1)%nscratch];
        U2 = buffers_U[(n+2)%nscratch];
        
        // Shifted time
        t = n*dt-2.0*td;
        pi2fm2t2 = pi*pi*fm*fm*t*t;
        
        // Set kernel arguments
        h_errchk(
            clSetKernelArg(kernel, 0, sizeof(cl_mem), &U0 ),
            "setting kernel argument 0"
        );
        h_errchk(
            clSetKernelArg(kernel, 1, sizeof(cl_mem), &U1 ),
            "setting kernel argument 1"
        );
        h_errchk(
            clSetKernelArg(kernel, 2, sizeof(cl_mem), &U2 ),
            "setting kernel argument 2"
        );
        h_errchk(
            clSetKernelArg(kernel, 11, sizeof(cl_float), &pi2fm2t2 ),
            "setting kernel argument 11"
        );
        
        // Enqueue the wave solver    
        h_errchk(
            clEnqueueNDRangeKernel(
                compute_queue,
                kernel,
                work_dim,
                NULL,
                global_size,
                local_size,
                0,
                NULL,
                &events[n%nscratch]), 
            "Running the kernel"
        );
          
        // Read memory from the buffer to the host in an asynchronous manner
        if (n>0) {
            cl_int copy_index=n-1;
            h_errchk(
                clEnqueueReadBuffer(
                    command_queues[copy_index%nscratch],
                    buffers_U[copy_index%nscratch],
                    blocking,
                    0,
                    nbytes_U,
                    &array_out[copy_index*N0*N1],
                    1,
                    &events[copy_index%nscratch],
                    NULL), 
                "Asynchronous copy from U2 on device to host"
            );
        }
    }
```

#### Make and run

In [6]:
!make

make: Nothing to be done for 'all'.


In [8]:
# Run the application
subprocess.run([os.path.join(os.getcwd(),"wave2d_async.exe"), "-gpu"])

	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
dt=0.001166, Vmax=343.000000
dt=0.00116618, fm=34.3, Vmax=343, dt2=1.35998e-06
The asynchronous calculation took 54295 milliseconds.

CompletedProcess(args=['/home/toby/Pelagos/Projects/OpenCL_Course/course_material/L9_IO_Optimisation/wave2d_async.exe', '-gpu'], returncode=0)

If we check the time elapsed we find that the concurrent IO solution took less time than the sequential IO solution. A trace of the OpenCL activity shows that IO is taking place during kernel execution.

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/asynchronous_io.png"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Concurrent IO solution.</figcaption>
</figure>


### Plot the wavefield and explore results

In [9]:
# Read the outputfile back in for display
output=np.fromfile("array_out.dat", dtype=float_type)
nimages=int(output.size//(defines["N0"]*defines["N1"]))
images=output.reshape(nimages, defines["N0"], defines["N1"])

py_helper.plot_slices(images)

interactive(children=(IntSlider(value=0, description='n', max=639), Output()), _dom_classes=('widget-interact'…

## Summary of learnings

In this module we explored how IO can take place at the same time as a kernel using multiple command queues. The concurrent IO solution was faster than the sequential IO solution. One does need to be careful that buffers are not being read or written at the same time as they are being used in a kernel, otherwise this leads to undefined behaviour. Events and the use of the **[clFinish](https://registry.khronos.org/OpenCL/sdk/3.0/docs/man/html/clFinish.html)** function can establish and enforce dependencies between activity that occurs across multiple command queues.

<address>
Written by Dr. Toby Potter of <a href="https://www.pelagos-consulting.com">Pelagos Consulting and Education</a> for the Pawsey Supercomputing Centre
</address>